**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]

# Lab 2: Nim game

The game of Nim is a well-known **combinatorial misère game** in which we have a number of objects (say coins 🪙 in this case) arranged in different rows. In each turn, a player can remove one or more objects from a single row and the player who takes the last object loses. The game is usually played with a number of rows and a maximum number of objects that can be removed in a turn. 

| Row | Objects |
| --- | --- |
| 0 | 🪙 |
| 1 | 🪙 🪙 🪙 |
| 2 | 🪙 🪙 🪙 🪙 🪙 |


### Task

Our goal is to write an agent able to play Nim with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn. Consider these types of agents:
1. An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
2. An agent using evolved rules using ES

### Instructions
* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)


In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np

## Nim class
First of all, we need to create a class that represents the game of Nim. This class will be used by the agents to play the game. Its attributes are:
- `rows`: the number of rows of the game
- `k`: the maximum number of objects that can be removed in a turn

Each move is represented by the tuple **Nimply** = `(row, num_objects)` and it is applied to the game by the method `nimming`.

In [ ]:
# Define a named tuple to represent a move
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        """ Initialize the Nim game with num_rows, each row has 2i+1 objects """
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        """ Return True if the game is still on-going """
        return sum(self._rows) > 0

    def __str__(self):
        """ Return a string representation of the game state """
        str = ""
        for i, row in enumerate(self._rows):
            str += f"Row {i}: " + "🪙" * row + "\n"
        return str

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        """ Apply the move and update the game state """
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

### Nim-sum
At any state of the game, we can define the **nim-sum** as the cumulative XOR value of the number of coins in each row. 

| Row | Objects | Binary |
| --- | --- | --- |
| 0 | 🪙 | 001 |
| 1 | 🪙 🪙 🪙 | 011 |
| 2 | 🪙 🪙 🪙 🪙 🪙 | 111 |

`nim-sum` = 001 XOR 011 XOR 111 = 101 = 5

##### Winning strategy
If the nim-sum is non-zero and the player to move is playing optimally, then he's guaranteed to win. 
- `nim-sum` = 0 $\implies$ losing position
- `nim-sum` != 0 $\implies$ winning position

> Therefore, the optimal strategy is to always leave the nim-sum equal to zero for the opponent.

e.g. Nimply = (2, 3) applied to the previous game

| Row | Objects | Binary |
| --- | --- | --- |
| 0 | 🪙 | 001 |
| 1 | 🪙 🪙 🪙 | 011 |
| 2 | 🪙 🪙 | 010 |

`nim-sum` = 001 XOR 011 XOR 010 = 000 = 0

> For any state with `nim-sum != 0` there could be multiple optimal moves, but we just need to find one to be sure to win. 

I implemented the following algorithm:
1. Compute the `nim-sum` of the current state
2. Find a row such that `nim-sum` XOR `num-objects` < `num-objects` (i.e. there are enough objects to remove to make the `nim-sum` equal to zero)
3. Return the Nimply = (row, `num-objects` - (`nim-sum` XOR `num-objects`))

In [ ]:
def nim_sum(state: Nim) -> int:

    # Convert the number of objects in each row to binary representation
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])

    # Perform bitwise XOR operation along the columns (axis=0)
    xor = tmp.sum(axis=0) % 2

    # Convert the binary representation back to integer
    return int("".join(str(_) for _ in xor), base=2)


def optimal_move(state: Nim) -> Nimply:
    """ Return the optimal move for the current state """
    
    # Compute the nim sum of the current state
    curr_nim_sum = nim_sum(state)

    # If the nim sum is 0, we are in a losing state :/
    # => Find the first non-empty row and remove 1 object
    #    We want the game to last as long as possible, in the hope that the opponent makes a mistake
    if curr_nim_sum == 0:
        for row, num_objects in enumerate(state.rows):
            if num_objects > 0:
                return Nimply(row, 1)
    
    # Otherwise, we find a row and a value that will make the nim sum 0
    # And the win is guaranteed! ;)
    for row, num_objects in enumerate(state.rows):
        xor_value = curr_nim_sum ^ num_objects
        if xor_value < num_objects:
            return Nimply(row, num_objects - xor_value)

    # If we reach here, something is wrong :'(
    raise ValueError("No valid move found.")


## Agent 1: expert system